In [1]:
import cv2
import numpy as np
import torch
from depth_anything_v2.dpt import DepthAnythingV2
import vtk
from PIL import Image
import math

xFormers not available
xFormers not available


In [2]:



import cv2
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
inpainting = pipeline(Tasks.image_inpainting, model='damo/cv_fft_inpainting_lama')
def refine_pic(input_location,output_path):
    input_mask_location = input_location.replace(".png","_mask.png")

    input = {
            'img':input_location,
            'mask':input_mask_location,
    }

    
    result = inpainting(input)
    vis_img = result[OutputKeys.OUTPUT_IMG]
    cv2.imwrite(output_path, vis_img)

2024-10-14 01:19:20,630 - modelscope - WARNING - Authentication has expired, please re-login with modelscope login --token "YOUR_SDK_TOKEN" if you need to access private models or datasets.
2024-10-14 01:19:21,815 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2024-10-14 01:19:22,330 - modelscope - INFO - initiate model from C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama
2024-10-14 01:19:22,346 - modelscope - INFO - initiate model from location C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama.
2024-10-14 01:19:22,347 - modelscope - INFO - initialize model from C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama
2024-10-14 01:19:22,347 - modelscope - INFO - BaseInpaintingTrainingModule init called, predict_only is False
2024-10-14 01:19:22,801 - modelscope - INFO - BaseInpaintingTrainingModule init done
2024-10-14 01:19:22,801 - modelscope - INFO - loading pretrained model from C:\Users\A

Loading weights for net_encoder


2024-10-14 01:19:23,118 - modelscope - WARNING - No preprocessor field found in cfg.
2024-10-14 01:19:23,118 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-10-14 01:19:23,118 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\Administrator\\.cache\\modelscope\\hub\\damo\\cv_fft_inpainting_lama'}. trying to build by task and model information.
2024-10-14 01:19:23,118 - modelscope - WARNING - No preprocessor key ('FFTInpainting', 'image-inpainting') found in PREPROCESSOR_MAP, skip building preprocessor.
2024-10-14 01:19:23,149 - modelscope - INFO - loading model from dir C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama
2024-10-14 01:19:23,150 - modelscope - INFO - BaseInpaintingTrainingModule init called, predict_only is True
2024-10-14 01:19:23,280 - modelscope - INFO - BaseInpaintingTrainingModule init done
2024-10

In [3]:
from PIL import Image, ImageEnhance

def adjust_contrast_if_brightness_high(image_path, output_path, brightness_threshold=0.1, contrast_factor=1000):
    # 打开图片
    img = Image.open(image_path)

    # 计算图片亮度
    brightness_enhancer = ImageEnhance.Brightness(img)
    brightness = brightness_enhancer.enhance(1.0)
    brightness_value = brightness.convert('L').point(lambda x: x / 255.0, 'F').getextrema()[1]

    # 如果亮度大于阈值，增加对比度
    if brightness_value > brightness_threshold:
        contrast_enhancer = ImageEnhance.Contrast(img)
        img = contrast_enhancer.enhance(contrast_factor)
    
    # 保存调整后的图片
    img.save(output_path)

# 示例使用
# input_image = 'view_2_mask.png'   # 输入图片路径
# output_image = 'view_2_mask.png' # 输出图片路径
# adjust_contrast_if_brightness_high(input_image, output_image)


In [13]:

model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vitb.pth', map_location='cuda:0')) #cpu


# model = DepthAnythingV2(encoder='vits', features=64, out_channels=[48, 96, 192, 384])
# model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vits.pth', map_location='cuda:0'))

def pic23d(pic_name,path,depth_hight = 10):
    model.cuda().eval()   #   cpu model.eval() 
    if ".png"in pic_name or ".webp" in pic_name:
        image = Image.open(pic_name)  # 或者 "example.png"

        # 转换并保存为 JPG 格式
        pic_name = pic_name.replace(".webp",".jpg").replace(".png",".jpg")
        image.convert("RGB").save(pic_name, "JPEG")
    raw_img = cv2.imread(path+pic_name)  
    height, width = raw_img.shape[:2]  
    target_height = 800  
    scale_ratio = target_height / height  
    target_width = int(width * scale_ratio)  
    raw_img = cv2.resize(raw_img, (target_width, target_height))  


    depth = model.infer_image(raw_img) # HxW raw depth map



    depth_normalized = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  
    depth_display = (depth_normalized * 255).astype(np.uint8)  # 转换为8位图像以便显示  
    cv2.imwrite('./depth_map.png', depth_display)  

    fx = 50*4  # 焦距x  
    fy = 50*4# 焦距y  
    cx = raw_img.shape[1] / 2.0  # 光心x  
    cy = raw_img.shape[0] / 2.0  # 光心y  
    height, width = raw_img.shape[:2]  
    
    # 推断深度图  
    depth_ = depth_display/255
    # 深度图可能需要归一化或转换到实际的深度值，这里假设它已经是所需的格式  
    
    # 创建一个空的数组来存储空间位置坐标  
    pos = np.zeros((height * width, 3), dtype=np.float32)  
    
    # 遍历深度图的每个像素  
    # Calculate indices for each pixel
    v_indices,u_indices = np.indices(depth_.shape)

    # Calculate spatial coordinates using vectorized operations
    X = (u_indices - cx)*(3-depth_) / fx
    Y = (v_indices - cy)*(3-depth_) / fy
    Z = depth_ * depth_hight

    # Stack the coordinates into a single array
    pos = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)
    
    # 现在pos数组包含了深度图上每个点的空间位置坐标
    # 计算面的面积
    def calculate_face_areas(pos, indices):
        areas = []
        for idx in indices:
            p0, p1, p2, p3 = pos[idx]
            # 计算两个三角形的面积
            v1 = p1 - p0
            v2 = p2 - p0
            v3 = p3 - p0
            area1 = np.linalg.norm(np.cross(v1, v2)) / 2.0
            area2 = np.linalg.norm(np.cross(v2, v3)) / 2.0
            areas.append(area1 + area2)
        return np.array(areas)
    all_face = False
    if  all_face == True:
        with open("a.obj","w") as t:
            t.writelines("mtllib my_mtl.mtl"+"\n")
                
            for l in pos:
                L = "v "+str(l[0])+" "+str(l[1])+" "+str(l[2])+"\n"
                t.writelines(L )
            
            w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
            h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
            t_array_ = np.stack([w_x,h_y],axis=1)


            for i,j in t_array_:
                L = "vt " +str(i)+" "+str(j) +"\n"
                t.writelines(L )                
        #  
        #          
            t.writelines("\nusemtl my_mtl\n" )              
            b = np.array(range(height * width)).reshape([ height ,width])
            print(b.shape)  


            for i in range(1,b.shape[0]-1):
                for j in range(1,b.shape[1]-1):
                    p = [ b[i,j] , b[i+1,j],b[i+1,j+1],b[i,j+1]]
            
                    L = "f "+str(p[0])+"/"+ str(p[0])+" "+str(p[1])+"/"+ str(p[1])+" "+str(p[2])+"/"+ str(p[2])+" "+str(p[3])+"/"+ str(p[3])+"\n"
                    t.writelines(L )


    else:

    #############################
        # 在生成面之后添加以下代码
        # 假设 b 是面索引的二维数组
        w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
        h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
        t_array_ = np.stack([w_x,h_y],axis=1)

        b = np.array(range(height * width)).reshape([ height ,width])
        face_indices = []
        for i in range(1, b.shape[0]-1):
            for j in range(1, b.shape[1]-1):
                p = [b[i,j], b[i+1,j], b[i+1,j+1], b[i,j+1]]
                face_indices.append(p)

        face_indices = np.array(face_indices)

        # 计算面面积
        areas = calculate_face_areas(pos, face_indices)

        # 删除面积大于0.02的面
        valid_indices = areas <= 0.01
        filtered_face_indices = face_indices[valid_indices]

        # 更新OBJ文件生成
        with open("filtered_model.obj", "w") as t:
            t.writelines("mtllib my_mtl.mtl" + "\n")

            for l in pos:
                L = "v " + str(l[0]) + " " + str(l[1]) + " " + str(l[2]) + "\n"
                t.writelines(L)

            # UV 坐标
            for i, j in t_array_:
                L = "vt " + str(i) + " " + str(j) + "\n"
                t.writelines(L)

            t.writelines("\nusemtl my_mtl\n")

            # 重新写入面
            for idx in filtered_face_indices:
                p = [idx[0], idx[1], idx[2], idx[3]]
                L = "f " + " ".join(f"{i + 1}/{i + 1}" for i in p) + "\n"  # 注意 OBJ 文件索引从1开始
                t.writelines(L)

#######################

        valid_indices = areas > 0.01
        filtered_face_indices = face_indices[valid_indices]

        # 更新OBJ文件生成
        with open("filtered_model_2.obj", "w") as t:
            t.writelines("mtllib green_mtl.mtl" + "\n")

            for l in pos:
                L = "v " + str(l[0]) + " " + str(l[1]) + " " + str(l[2]) + "\n"
                t.writelines(L)

            # UV 坐标
            for i, j in t_array_:
                L = "vt " + str(i) + " " + str(j) + "\n"
                t.writelines(L)

            t.writelines("\nusemtl green_mtl\n")

            # 重新写入面
            for idx in filtered_face_indices:
                p = [idx[0], idx[1], idx[2], idx[3]]
                L = "f " + " ".join(f"{i + 1}/{i + 1}" for i in p) + "\n"  # 注意 OBJ 文件索引从1开始
                t.writelines(L)
                
    with open("my_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl my_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd {pic_name}
    """
        )


    with open("green_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl green_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd {pic_name}
    """
        )



    def render_and_save_images(obj_files, texture_files,mask="", image_size=(800, 800)):
        # 读取模型

        # 创建渲染窗口
        render_window = vtk.vtkRenderWindow()       
        render_window.SetSize(image_size[0], image_size[1])  # 设置渲染窗口大小 
 
        # 创建左右视角的渲染器
        total_pic_count = 36
        renderers = []
        for i in range(total_pic_count):
            renderer = vtk.vtkRenderer()
            render_window.AddRenderer(renderer)
            renderers.append(renderer)
            # 禁用光照
            renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光

            for obj_file, pic_name in zip(obj_files, texture_files):
                # 读取模型
                reader = vtk.vtkOBJReader()
                reader.SetFileName(obj_file)
                reader.Update()
                
                # 创建一个映射器
                mapper = vtk.vtkPolyDataMapper()
                mapper.SetInputConnection(reader.GetOutputPort())
                
                # 创建一个演员
                actor = vtk.vtkActor()
                actor.SetMapper(mapper)
                
                # 读取贴图文件
                texture = vtk.vtkTexture()
                texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
                texture_reader.SetFileName(pic_name)  # 根据.mtl文件中的路径设置
                texture_reader.Update()
                texture.SetInputConnection(texture_reader.GetOutputPort())
                
                # 将贴图应用到演员
                actor.SetTexture(texture)
                renderer.AddActor(actor)


                actor.GetProperty().SetLighting(False)
            # 设置摄像机
            camera = vtk.vtkCamera()
            
            camera.SetFocalPoint(0, 0, 0)
            # if i == 0:  # 左视角
            #     camera.SetPosition(-2, 0, 30)  # 可以根据需要调整位置
            #     camera.SetViewUp(0, -1, 0)
            # else:  # 右视角
            #     camera.SetPosition(2, 0, 30)  # 右视角位置
            #     camera.SetViewUp(0, -1, 0)

      
            camera.SetPosition(-18+i*1, 0, 25)  # 可以根据需要调整位置
            camera.SetViewUp(0, -1, 0)

            renderer.SetActiveCamera(camera)

            # 添加演员到渲染器
            renderer.AddActor(actor)
            renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

            # 渲染
            render_window.Render()

            # 保存图像
            window_to_image_filter = vtk.vtkWindowToImageFilter()
            window_to_image_filter.SetInput(render_window)
            window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
            window_to_image_filter.Update()

            writer = vtk.vtkPNGWriter()
            writer.SetFileName(f'view_{i}'+mask+'.png')
            writer.SetInputConnection(window_to_image_filter.GetOutputPort())
            # writer.SetCompression(0)  # 设置为0表示无压缩，1表示有压缩（PNG默认是压缩）
            writer.Write()
            if mask!="":
                adjust_contrast_if_brightness_high(f'view_{i}'+mask+'.png', f'view_{i}'+mask+'.png')

                refine_pic(f'view_{i}'+'.png',f'view_{i}'+'.png')

        # 合并图片
        # images = [Image.open(f'view_{i}.png') for i in [0,total_pic_count-1]]
        # widths, heights = zip(*(i.size for i in images))

        # total_width = sum(widths)
        # max_height = max(heights)

        # new_image = Image.new('RGB', (total_width, max_height))

        # x_offset = 0
        # for img in images:
        #     new_image.paste(img, (x_offset, 0))
        #     x_offset += img.width

        # new_image.save(output_file)



        # 图片序列
        image_files = [f"view_{i}.png" for i in range(total_pic_count)]  # ["view_0.png", "view_1.png", ..., "view_9.png"]

        # 打开所有图片
        images = [Image.open(image_file) for image_file in image_files]

        # 将图片转换为 GIF
        images[0].save(
            'output.gif',
            save_all=True,
            append_images=images[1:],
            duration=100,  # 每帧持续时间，单位为毫秒
            loop=0  # 0 表示无限循环
    )
        
    obj_files = [ 'filtered_model_2.obj','filtered_model.obj']
    texture_files = [pic_name,pic_name, ]  # 贴图文件名
    render_and_save_images(obj_files, texture_files)
    # texture_files = ['write_texture.jpg','black_texture.jpg', ]  # 贴图文件名
    # render_and_save_images(obj_files, texture_files,mask="_mask")

path = "./"
pic_name = "Q10.png"
pic23d(pic_name,path,10)

In [20]:
import trimesh
import pymeshfix
import numpy as np

def fill_holes_in_obj(input_obj_path, output_obj_path):
    # Step 1: Load the .obj file using trimesh
    mesh = trimesh.load(input_obj_path, force='mesh')

    # Check if the mesh has UV mapping and materials
    # if not mesh.visual.uv:
    #     raise ValueError("The model does not contain UV mapping")

    # # Extract the original UV coordinates and material info
    # original_uv = mesh.visual.uv
    # original_materials = mesh.visual.material

    # Convert trimesh mesh to pymeshfix format
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)

    # Step 2: Use pymeshfix to fill holes
    meshfix = pymeshfix.MeshFix(vertices, faces)
    meshfix.repair(verbose=True, joincomp=True, remove_smallest_components=False)

    # Get the repaired mesh data
    fixed_vertices = meshfix.v
    fixed_faces = meshfix.f

    # Step 3: Create a new trimesh object with fixed geometry
    fixed_mesh = trimesh.Trimesh(vertices=fixed_vertices, faces=fixed_faces)

    # Step 4: Re-apply original UV mapping and material
    # fixed_mesh.visual.uv = original_uv
    # fixed_mesh.visual.material = original_materials

    # Step 5: Export the fixed .obj file
    fixed_mesh.export(output_obj_path)
    print(f"Fixed .obj file saved to {output_obj_path}")

# Example usage
input_obj_path = 'filtered_model.obj'  # Replace with your input .obj file path
output_obj_path = 'output_model_fixed.obj'  # Replace with your output .obj file path
fill_holes_in_obj(input_obj_path, output_obj_path)


Patching holes...
Patched 48 holes
Fixing degeneracies and intersections
Fixed .obj file saved to output_model_fixed.obj


In [16]:
!pip install pymeshfix

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.4 MB 2.0 MB/s eta 0:00:01
     -- ------------------------------------- 0.1/1.4 MB 2.0 MB/s eta 0:00:01
     -- ------------------------------------- 0.1/1.4 MB 2.0 MB/s eta 0:00:01
     ----- ---------------------------------- 0.2/1.4 MB 871.5 kB/s eta 0:00:02
     ------- -------------------------------- 0.3/1.4 MB 1.1 MB/s eta 0:00:01
     ---------- ----------------------------- 0.4/1.4 MB 1.4 MB/s eta 0:00:01
     ----------- ---------------------------- 0.4/1.4 MB 1.2 MB/s eta 0:00:01
     --------------- ------------------------ 0.5/1.4 MB 1.4 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.4 MB 1.5 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.4 MB 1.5 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.4 MB 1.5 MB/s eta 0:00:01
     ---

In [8]:
# import vtk

# def render_and_save_images(obj_files, pic_names, output_prefix):
#     # 创建渲染窗口
#     render_window = vtk.vtkRenderWindow()
    
#     # 创建左右视角的渲染器
#     total_pic_count = 36
#     renderers = []
    
#     for i in range(total_pic_count):
#         renderer = vtk.vtkRenderer()
#         render_window.AddRenderer(renderer)
#         renderers.append(renderer)
        
#         # 禁用光照
#         renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光
        
#         for obj_file, pic_name in zip(obj_files, pic_names):
#             # 读取模型
#             reader = vtk.vtkOBJReader()
#             reader.SetFileName(obj_file)
#             reader.Update()
            
#             # 创建一个映射器
#             mapper = vtk.vtkPolyDataMapper()
#             mapper.SetInputConnection(reader.GetOutputPort())
            
#             # 创建一个演员
#             actor = vtk.vtkActor()
#             actor.SetMapper(mapper)
            
#             # 读取贴图文件
#             texture = vtk.vtkTexture()
#             texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
#             texture_reader.SetFileName(pic_name)  # 根据.mtl文件中的路径设置
#             texture_reader.Update()
#             texture.SetInputConnection(texture_reader.GetOutputPort())
            
#             # 将贴图应用到演员
#             actor.SetTexture(texture)
#             renderer.AddActor(actor)
        
#         # 设置摄像机
#         camera = vtk.vtkCamera()
#         camera.SetFocalPoint(0, 0, 0)
#         camera.SetPosition(-18 + i * 1, 0, 30)  # 可以根据需要调整位置
#         camera.SetViewUp(0, -1, 0)
#         renderer.SetActiveCamera(camera)

#         # 设置背景颜色
#         renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

#         # 渲染
#         render_window.Render()

#         # 保存图像
#         window_to_image_filter = vtk.vtkWindowToImageFilter()
#         window_to_image_filter.SetInput(render_window)
#         window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
#         window_to_image_filter.Update()

#         writer = vtk.vtkPNGWriter()
#         writer.SetFileName(f'{output_prefix}_view_{i}.png')
#         writer.SetInputConnection(window_to_image_filter.GetOutputPort())
#         writer.Write()

# # 使用示例
# obj_files = ['filtered_model.obj', 'filtered_model_2.obj']
# pic_names = ['Q8.jpg', 'green_texture.jpg']  # 对应的贴图文件
# render_and_save_images(obj_files, pic_names, 'output')

In [3]:
# import cv2
# from modelscope.outputs import OutputKeys
# from modelscope.pipelines import pipeline
# from modelscope.utils.constant import Tasks

# input_location = './view_2.png'
# input_mask_location = './view_2_mask.png'

# input = {
#         'img':input_location,
#         'mask':input_mask_location,
# }

# inpainting = pipeline(Tasks.image_inpainting, model='damo/cv_fft_inpainting_lama', refine=True)
# result = inpainting(input)
# vis_img = result[OutputKeys.OUTPUT_IMG]
# cv2.imwrite('result.png', vis_img)



2024-10-10 02:22:07,825 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2024-10-10 02:22:08,191 - modelscope - INFO - initiate model from C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama
2024-10-10 02:22:08,191 - modelscope - INFO - initiate model from location C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama.
2024-10-10 02:22:08,191 - modelscope - INFO - initialize model from C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama
2024-10-10 02:22:08,236 - modelscope - INFO - BaseInpaintingTrainingModule init called, predict_only is False


Loading weights for net_encoder


2024-10-10 02:22:09,441 - modelscope - INFO - BaseInpaintingTrainingModule init done
2024-10-10 02:22:09,441 - modelscope - INFO - loading pretrained model from C:\Users\Administrator\.cache\modelscope\hub\damo\cv_fft_inpainting_lama\pytorch_model.pt
2024-10-10 02:22:10,245 - modelscope - WARNING - No preprocessor field found in cfg.
2024-10-10 02:22:10,245 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-10-10 02:22:10,245 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'C:\\Users\\Administrator\\.cache\\modelscope\\hub\\damo\\cv_fft_inpainting_lama'}. trying to build by task and model information.
2024-10-10 02:22:10,245 - modelscope - WARNING - No preprocessor key ('FFTInpainting', 'image-inpainting') found in PREPROCESSOR_MAP, skip building preprocessor.
2024-10-10 02:22:10,261 - modelscope - INFO - loading model from dir C:\Users\Administrator\

True